##### F1 Capstone Project

What is the goal?  
- To see whether, by giving the model data up to x year, the model can predict the race and/or championship results of x year.
- Within the current season (/and past seasons?), to build an app predicting the finishing result of each driver + team by giving the model data up to x race within the season.

Considerations:  
- should data be unmarried from specific drivers for better generalisation, or does that remove a crucial factor?

Challenges:  
- some years have seen big regulation changes
- best predictors of race results come from pre-race sessions (esp. quali), so how to incorporate that

useful background resources:
[https://github.com/ethan-eplee/HorseRacePrediction](https://github.com/ethan-eplee/HorseRacePrediction)
[https://www.kaggle.com/discussions/general/333090](https://www.kaggle.com/discussions/general/333090)
[https://medium.com/@fernando.siguenza/building-an-ai-to-predict-f1-race-outcomes-a-data-science-journey-7f55e0d75b1e](https://medium.com/@fernando.siguenza/building-an-ai-to-predict-f1-race-outcomes-a-data-science-journey-7f55e0d75b1e)

In [1]:
import pandas as pd

Context and prep:  
- scoring was funky pre 1991, so for end-year WDC title, will use at earliest 1991-onwards data
- a lot of the necessary data was pulled during the midcourse, some cleanup + EDA performed

### will need 2025 data as the season progresses

In [80]:
#races_df: contains info of all GP 1950-2024 w/ geo info
races_df = pd.read_csv('../data/races_df.csv')
#results_df: contains placement of every driver at the conclusion of every GP (does not currently include sprint results for 2021-present), 
#but YTD points should include fastest lap for 2019-2024 <-- check this
results_df = pd.read_csv('../data/results_df.csv', dtype={'number': str})
#results from all 18 sprints, maybe add to model
sprints = pd.read_csv('../data/sprint_results.csv')

In [71]:
pd.read_csv('../data/results.csv')
#grab 'fastestLapTime', 'fastestLap', and 'raceId', 'driverId' for merge for results_df
fast_laps = pd.read_csv('../data/results.csv', usecols=['raceId', 'driverId', 'fastestLapTime', 'fastestLap'])

In [81]:
#finishPosition - '\N' if driver DNFed (not a NaN)
#finishPosNum - same as above, but floats w/ NaNs <-- REMOVE
#positionText - 'R' if driver DNFed
#positionOrder - lists all drivers in points order, including DNFs
results_df.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,finishPosition,positionText,positionOrder,points,...,url,birthYear,year,round,circuitId,GPname,date,startTime,driverName,finishPosNum
7880,7881,356,138,32,12,21,8,8,8,0.0,...,http://en.wikipedia.org/wiki/Satoru_Nakajima,1953,1989,1,36,Brazilian Grand Prix,1989-03-26,\N,Satoru Nakajima,8.0
1704,1705,99,32,19,15,13,11,11,11,0.0,...,http://en.wikipedia.org/wiki/Christian_Klien,1983,2004,10,8,French Grand Prix,2004-07-04,\N,Christian Klien,11.0
18044,18045,744,492,170,66,0,\N,W,38,0.0,...,http://en.wikipedia.org/wiki/Menato_Boffa,1930,1961,7,14,Italian Grand Prix,1961-09-10,\N,Menato Boffa,NaN
4005,4006,207,30,6,5,3,2,2,2,6.0,...,http://en.wikipedia.org/wiki/Michael_Schumacher,1969,1997,1,1,Australian Grand Prix,1997-03-09,\N,Michael Schumacher,2.0
22549,22553,926,826,9,26,12,\N,W,16,0.0,...,http://en.wikipedia.org/wiki/Daniil_Kvyat,1994,2015,1,1,Australian Grand Prix,2015-03-15,05:00:00,Daniil Kvyat,NaN


In [83]:
#dropping 'finishPosNum' because want placement to be int (I think?), and adding fastest laps
results_df = results_df.drop(columns = 'finishPosNum')
results_df = pd.merge(results_df, fast_laps, on = ['raceId', 'driverId'])

In [84]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26953 entries, 0 to 26952
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   resultId               26953 non-null  int64  
 1   raceId                 26953 non-null  int64  
 2   driverId               26953 non-null  int64  
 3   constructorId          26953 non-null  int64  
 4   number                 26953 non-null  object 
 5   grid                   26953 non-null  int64  
 6   finishPosition         26953 non-null  object 
 7   positionText           26953 non-null  object 
 8   positionOrder          26953 non-null  int64  
 9   points                 26953 non-null  float64
 10  laps                   26953 non-null  int64  
 11  time                   26953 non-null  object 
 12  statusId               26953 non-null  int64  
 13  status                 26953 non-null  object 
 14  constructorRef         26953 non-null  object 
 15  na

What features are important and can they be factored into the model?  
- performance in previous race(s) and pre-race sessions
- constructor? (good car = good performance)
- within teams, driver vs driver?
- team pitstop performance <- need to add
- weather (rain / rain expected)